In [51]:
!pip install -q torch torchvision torchaudio
!pip install -q transformers datasets accelerate Pillow huggingface_hub


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [52]:
import os
import json
import torch
import torch.nn as nn
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from transformers import CLIPVisionModel, AutoImageProcessor
from datasets import load_dataset
from huggingface_hub import snapshot_download

In [53]:
# --- DATALOADER ДЛЯ VQA-RAD (ЧЕРЕЗ HF DATASETS) ---
class VqaradDataset(Dataset):
    def __init__(self, image_processor, split='train', hf_repo_id='flaviagiammarino/vqa-rad'):
        self.image_processor = image_processor
        self.dataset = load_dataset(hf_repo_id, split=split, streaming=False)
        self.dataset = list(self.dataset)  # для небольшого датасета допустимо

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        item = self.dataset[idx]
        image = item['image'].convert("RGB")
        processed_output = self.image_processor(image, return_tensors="pt")

        if hasattr(processed_output, 'pixel_values'):
            processed_image = processed_output.pixel_values
        else:
            processed_image = processed_output

        if processed_image.dim() == 4:
            processed_image = processed_image.squeeze(0)

        return {
            "image": processed_image,
            "question": item['question'],
            "answer": item['answer']
        }

In [54]:
# --- DATALOADER ДЛЯ SLAKE ---
class SlakeDataset(Dataset):
    def __init__(self, image_processor, split='train', hf_repo_id='BoKelvin/SLAKE'):
        self.image_processor = image_processor
        self.root_dir = snapshot_download(repo_id=hf_repo_id, repo_type='dataset')
        img_dir_path = os.path.join(self.root_dir, 'imgs')
        if not os.path.exists(img_dir_path):
            os.system(f"unzip -q -o {os.path.join(self.root_dir, 'imgs.zip')} -d {self.root_dir}")
        json_path = os.path.join(self.root_dir, f"{split}.json")
        with open(json_path, 'r', encoding='utf-8') as f:
            full_data = json.load(f)
        self.dataset = [item for item in full_data if item['q_lang'] == 'en']

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        item = self.dataset[idx]
        image_name = item['img_name']
        image_name = image_name.split('/')
        image_name = os.path.join(image_name[0], image_name[1])
        image_path = os.path.join(self.root_dir, 'imgs', image_name)
        image = Image.open(image_path).convert("RGB")
        
        processed_output = self.image_processor(image, return_tensors="pt")
        if hasattr(processed_output, 'pixel_values'):
            processed_image = processed_output.pixel_values
        else:
            processed_image = processed_output

        if processed_image.dim() == 4:
            processed_image = processed_image.squeeze(0)

        return {
            "image": processed_image,
            "question": item['question'],
            "answer": item['answer']
        }

In [55]:
# --- 3. КЛАСС МОДЕЛИ (encoder + MLP)---
class VisionMLP(nn.Module):
    def __init__(self, vision_encoder, encoder_output_dim, mlp_output_dim, hidden_dim=2560):
        super().__init__()
        self.vision_encoder = vision_encoder
        
        # MLP для обработки последовательности патчей
        self.mlp = nn.Sequential(
            nn.Linear(encoder_output_dim, hidden_dim),
            nn.GELU(),
            nn.Linear(hidden_dim, mlp_output_dim)
        )

    def forward(self, pixel_values):
        outputs = self.vision_encoder(pixel_values, output_hidden_states=False)
        # Берём ВСЕ ПАТЧИ (без [CLS] токена)
        patch_tokens = outputs.last_hidden_state[:, 1:, :]  # (batch, num_patches, hidden_dim)
        return self.mlp(patch_tokens)  # (batch, num_patches, mlp_output_dim)


def get_model(encoder_choice, mlp_output_dim, hidden_dim=2560):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model_paths = {
        'standard_clip': 'openai/clip-vit-base-patch32',
        'pubmedclip': 'flaviagiammarino/pubmed-clip-vit-base-patch32',
        # Примеры других CLIP-моделей:
        # 'biomed_clip': 'microsoft/BiomedCLIP-PubMedBERT_256-vit_base_patch16_224',  # если есть HF версия
        # 'large_clip': 'openai/clip-vit-large-patch14'
    }

    vision_encoder = CLIPVisionModel.from_pretrained(model_paths[encoder_choice])
    image_processor = AutoImageProcessor.from_pretrained(model_paths[encoder_choice])
    
    # Получаем размерность скрытого состояния
    encoder_output_dim = vision_encoder.config.hidden_size
    

    model = VisionMLP(
        vision_encoder=vision_encoder,
        encoder_output_dim=encoder_output_dim,
        mlp_output_dim=mlp_output_dim
    ).to(device)

    # Замораживаем CLIP (только обучаем MLP)
    for param in model.vision_encoder.parameters():
        param.requires_grad = False

    return model, image_processor

In [56]:
def get_dataloader(dataset_choice, image_processor, batch_size=4):
    if dataset_choice == 'slake':
        dataset = SlakeDataset(image_processor=image_processor, split='train')
    elif dataset_choice == 'vqa_rad':
        dataset = VqaradDataset(image_processor=image_processor, split='train')

    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=0)
    return dataloader

### Дальше тест что все работает, по сути нужная часть - следующая (получение замороженного энкодера с размороженным MLP и даталоадера):

    model, image_processor = get_model(encoder_choice=encoder_choice, mlp_output_dim=mlp_output_dim)
    dataloader = get_dataloader(dataset_choice=dataset_choice, image_processor=image_processor, batch_size=4)

In [67]:
def eval(encoder_choice, dataset_choice, mlp_output_dim=4096):
    device = "cuda" if torch.cuda.is_available() else "cpu"

    print("\n--- Обработка батча с ПАТЧАМИ ---")
    model, image_processor = get_model(encoder_choice=encoder_choice, mlp_output_dim=mlp_output_dim)
    dataloader = get_dataloader(dataset_choice=dataset_choice, image_processor=image_processor, batch_size=4)

    model.eval()
    with torch.no_grad():
        batch = next(iter(dataloader))
        images = batch['image'].to(device)
        image_embeddings = model(images)

        print(f"\n- Форма изображений: {images.shape}")
        print(f"- Форма патч-эмбеддингов: {image_embeddings.shape}")

        assert image_embeddings.dim() == 3, "Ожидалась 3D-форма для патчей!"
        assert image_embeddings.shape[-1] == mlp_output_dim, "Неверная размерность эмбеддинга!"
        
        expected_patches = (224 // 32) ** 2  # = 49

        print(f"- Патчей на изображение: {image_embeddings.shape[1]} (ожидалось ~{expected_patches})")
        print(image_embeddings[0][0])
    
    print("\n*** УСПЕХ: Получены эмбеддинги ВСЕХ ПАТЧЕЙ ***")

In [63]:
eval(encoder_choice='pubmedclip', dataset_choice='vqa_rad')


--- Обработка батча с ПАТЧАМИ ---

- Форма изображений: torch.Size([4, 3, 224, 224])
- Форма патч-эмбеддингов: torch.Size([4, 49, 4096])
- Патчей на изображение: 49 (ожидалось ~49)
tensor([-0.0026,  0.0096,  0.0352,  ...,  0.0510,  0.1204, -0.0607],
       device='cuda:0')

*** УСПЕХ: Получены эмбеддинги ВСЕХ ПАТЧЕЙ ***


In [64]:
eval(encoder_choice='standard_clip', dataset_choice='vqa_rad')


--- Обработка батча с ПАТЧАМИ ---

- Форма изображений: torch.Size([4, 3, 224, 224])
- Форма патч-эмбеддингов: torch.Size([4, 49, 4096])
- Патчей на изображение: 49 (ожидалось ~49)
tensor([-0.0485, -0.0370,  0.0320,  ...,  0.2097,  0.1049,  0.1029],
       device='cuda:0')

*** УСПЕХ: Получены эмбеддинги ВСЕХ ПАТЧЕЙ ***


In [65]:
eval(encoder_choice='pubmedclip', dataset_choice='slake')


--- Обработка батча с ПАТЧАМИ ---


Fetching 8 files:   0%|          | 0/8 [00:00<?, ?it/s]


- Форма изображений: torch.Size([4, 3, 224, 224])
- Форма патч-эмбеддингов: torch.Size([4, 49, 4096])
- Патчей на изображение: 49 (ожидалось ~49)
tensor([-0.0887,  0.0570, -0.0225,  ..., -0.1287,  0.0247,  0.2395],
       device='cuda:0')

*** УСПЕХ: Получены эмбеддинги ВСЕХ ПАТЧЕЙ ***


In [66]:
eval(encoder_choice='standard_clip', dataset_choice='slake')


--- Обработка батча с ПАТЧАМИ ---


Fetching 8 files:   0%|          | 0/8 [00:00<?, ?it/s]


- Форма изображений: torch.Size([4, 3, 224, 224])
- Форма патч-эмбеддингов: torch.Size([4, 49, 4096])
- Патчей на изображение: 49 (ожидалось ~49)
tensor([-0.0504, -0.1674, -0.0298,  ..., -0.0098, -0.1319,  0.0947],
       device='cuda:0')

*** УСПЕХ: Получены эмбеддинги ВСЕХ ПАТЧЕЙ ***
